In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts

hv.extension("matplotlib")

### Do Backprop with only Single Neuron Layers with Depth of 2 ( Stacked Linear Regression)

In [ ]:
# Generate data
np.random.seed(0)
n = 200

x = np.linspace(-10, 10, 1000)
noise = np.random.normal(size=len(x))

y = 3 * x + 4 + noise

actuals = hv.Scatter(zip(x, y))
actuals

In [ ]:
# Split into train and test sets.
np.random.seed(0)

# Create index for train and test that is randomly shuffled.
idx = np.arange(len(x))
np.random.shuffle(idx)
split_pct = 0.80
split_at = int(len(x) * split_pct)
idx_train, idx_test = idx[slice(split_at)], idx[slice(-split_at)]

# Split into train and test sets.
x_train, y_train = x[idx_train].copy(), y[idx_train].copy()
x_test, y_test = x[idx_test].copy(), y[idx_test].copy()

In [ ]:
# Create standard scalers for train x and y.
def standard_scaler(arr):
    mean = arr.mean()
    std = arr.std()

    def ss(vv):
        return (vv - mean) / std

    def ss_inv(vv):
        return (vv * std) + mean

    return ss, ss_inv


ss_x, ss_x_inv = standard_scaler(x_train)
ss_y, ss_y_inv = standard_scaler(y_train)

Given the cost function:
$J(\hat{y}) = \frac{1}{n}\sum_{i=0}^{n} (y_i - \hat{y}_i)^2$  
Where $\hat{y} = (W \cdot z + \beta)$  
Where $z$ is the previous activation in a layer and for $W_0 = X$

The gradient of the cost function $J(\hat{y})$ w.r.t. the Weights $W$ and Biases $\beta$ gives us
$$ \frac{dJ(\hat{y})}{dW} = \frac{-2}{n}\sum_{i=0}^{n} (y_i - \hat{y_i})z_i$$  
$$ \frac{dJ(\hat{y})}{d\beta} = \frac{-2}{n}\sum_{i=0}^{n} (y_i - \hat{y_i}) $$  

It's important to note that the loss w.r.t the weights of layer j $W_j$ are are not influences by layer j.  Rather they are influenced by the activiation directly prior to layer $W_j$  

In The case that there is only one layer of weights then the previous activation is just the input data $X$.
In other words such a scenario is just a linear regression.
$$ \frac{dJ(\hat{y})}{dW} = \frac{-2}{n}\sum_{i=0}^{n} (y_i - \hat{y_i})X_i$$  
$$ \frac{dJ(\hat{y})}{d\beta} = \frac{-2}{n}\sum_{i=0}^{n} (y_i - \hat{y_i}) $$  

In [ ]:
class StackedLinReg2Deep:
    def __init__(self, learning_rate=0.001):
        # Intialiaze learning rate.
        self.learning_rate = learning_rate

        # Intialize weights and biases.
        self.w0 = np.random.random(1)[0]
        self.b0 = np.random.random(1)[0]
        self.w1 = np.random.random(1)[0]
        self.b1 = np.random.random(1)[0]

        # Initialize loss.
        self.j = np.inf

        # Initialize gradients.
        self.dj_wrt_dw0 = 0
        self.dj_wrt_db0 = 0
        self.dj_wrt_dw1 = 0
        self.dj_wrt_db1 = 0

    # fwd computes each foward step through network.
    def fwd(self, x):
        z0 = x * self.w0 + self.b0
        z1 = z0 * self.w1 + self.b1
        return np.array([x, z0, z1])

    # bwd computes the gradient of the cost function given z (forward propogation
    # vector) and y.
    def bwd(self, z, y):
        # Split up z into each forward pass result.
        x, z0, z1 = z

        # Compute loss.
        # self.j = np.power(y - (self.w1 * (x * self.w0 + b0) + b1), 2).sum()
        self.j = np.power(y - z1, 2).mean()

        # Compute derivative of last layer.
        self.dj_wrt_dw1 = 2 * (z0 * (y - z1)).mean()
        self.dj_wrt_db1 = 2 * (y - z1).mean()

        # Calculate what the activation should have been for the last layer.
        z_hat = (y - self.b1) / self.w1

        self.dj_wrt_dw0 = 2 * (x * (z_hat - z0)).mean()
        # Compute derivative of first layer.
        self.dj_wrt_db0 = 2 * (z_hat - z0).mean()

    # step updates the weights and biases based from the gradient.
    def step(self):
        self.w0 += self.learning_rate * self.dj_wrt_dw0
        self.b0 += self.learning_rate * self.dj_wrt_db0
        self.w1 += self.learning_rate * self.dj_wrt_dw1
        self.b1 += self.learning_rate * self.dj_wrt_db1

In [ ]:
def train(model, x, y, epochs, early_stopping=False, epsilon=0.000001):
    # Initialize the previous loss for early stopping.
    loss_prev = model.j
    # fwd, bwd, and step for each epoch.
    for i in range(epochs):
        # Foward pass.
        z = model.fwd(x)

        # Backward pass.
        model.bwd(z, y)
        print(f"{i:08}> loss (j): {model.j:>7f}")

        # Setup early stopping.
        if (loss_prev < model.j or model.j < epsilon) and early_stopping:
            break
        loss_prev = model.j

        # Update weights and biases.
        model.step()

In [ ]:
def pred_plot(x, y, scale_x, scale_inv_pred, title, model):
    preds = scale_inv_pred(model.fwd(scale_x(x))[-1])
    return hv.Scatter(zip(x, y)) * hv.Scatter(zip(x, preds)).opts(title=title)

In [ ]:
np.random.seed(0)
stacked_lin_reg = StackedLinReg2Deep(learning_rate=0.2)
train(stacked_lin_reg, ss_x(x_train), ss_y(y_train), 20000, early_stopping=True)

In [ ]:
hv.Layout(
    [
        pred_plot(
            x_train, y_train, ss_x, ss_y_inv, "Train Predictions", stacked_lin_reg
        ),
        pred_plot(x_test, y_test, ss_x, ss_y_inv, "Test Predictions", stacked_lin_reg),
        pred_plot(x, y, ss_x, ss_y_inv, "All Predictions", stacked_lin_reg),
    ]
)

## Now to Generalize the Number of Layers

In [ ]:
class StackedLinReg:
    def __init__(self, n_layers, learning_rate=0.001):
        # Intialiaze learning rate.
        self.learning_rate = learning_rate

        # Intialize weights and biases.
        self.W = np.random.random(n_layers)
        self.B = np.random.random(n_layers)

        # Initialize loss.
        self.j = np.inf

        # Initialize gradients.
        self.dj_wrt_dW = np.zeros(n_layers)
        self.dj_wrt_db = np.zeros(n_layers)

    # fwd computes each foward step through network.
    def fwd(self, x):
        z = np.zeros((self.n_layers + 1, len(x)))
        z[0] = x
        for i in range(self.n_layers):
            z[i + 1] = z[i] * self.W[i] + self.B[i]
        return z

    # bwd computes the gradient of the cost function given z (forward propogation
    # vector) and y.
    def bwd(self, z, y):
        # Compute loss.
        self.j = np.power(y - z[-1], 2).mean()

        # z_hat is what the activation should have been.
        z_hat = y
        # Back propogate the activations through each layer.
        for i in range(self.n_layers):
            # Take the gradient with respect to the W and b
            self.dj_wrt_dW[i] = 2 * (z[i] * (z_hat - z[i + 1])).mean()
            self.dj_wrt_db[i] = 2 * (z_hat - z[i + 1]).mean()
            # Update correct activation given current layers' W and b.
            z_hat = (z_hat - self.B[i]) / self.W[i]

    # step updates the weights and biases based from the gradient.
    def step(self):
        self.W += self.learning_rate * self.dj_wrt_dW
        self.B += self.learning_rate * self.dj_wrt_db

    @property
    def n_layers(self):
        return len(self.W)

In [ ]:
np.random.seed(0)
stacked_lin_reg2 = StackedLinReg(3, learning_rate=0.05)
train(stacked_lin_reg2, ss_x(x_train), ss_y(y_train), 20000, early_stopping=True)

In [ ]:
hv.Layout(
    [
        pred_plot(
            x_train, y_train, ss_x, ss_y_inv, "Train Predictions", stacked_lin_reg2
        ),
        pred_plot(x_test, y_test, ss_x, ss_y_inv, "Test Predictions", stacked_lin_reg2),
        pred_plot(x, y, ss_x, ss_y_inv, "All Predictions", stacked_lin_reg2),
    ]
)

Given the cost function:  

$J(\hat{y}) = \frac{1}{n}\sum_{i=0}^{n} (y_i - \hat{y}_i)^2$  

Where $f$ is the activation function.  
Where $\hat{y} = f(W \cdot z + \beta)$  
Where $z$ is the previous activation in a layer and for $W_0 = X$

The gradient of the cost function $J(\hat{y})$ w.r.t. the Weights $W$ and Biases $\beta$ gives us
$$ \frac{dJ(\hat{y})}{dW} = \frac{-2}{n}\sum_{i=0}^{n} (y_i - \hat{y_i})f^{'}(z_i)z_i$$  
$$ \frac{dJ(\hat{y})}{d\beta} = \frac{-2}{n}\sum_{i=0}^{n} (y_i - \hat{y_i})f^{'}(z_i)$$  

Take the example of taking the gradient of the loss w.r.t. each weight $W$ in each layer for a NN with 3 layers 0, 1, and 2.

$$\hat{y} = z_2$$
$$z_2 = f( W_2 z_1 + \beta_2 )$$
$$z_1 = f( W_1 z_0 + \beta_1 )$$
$$z_0 = f( W_0 x + \beta_0 )$$

Layer2:  
$$ \frac{dJ(\hat{y})}{dW_2} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) z_1 $$

Layer1:  
$$ \frac{dJ(\hat{y})}{dW_1} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) ( W_2 ( f'( W_1 z_0 + \beta_1 ) z_0 ) ) $$

Layer0:  
$$ \frac{dJ(\hat{y})}{dW_1} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) ( W_2 ( f'( W_1 z_0 + \beta_1 ) ( w_1 f'( w_0 x_0 + b_0 ) x_0 ) ) ) $$


Organizing the derivaties differently we get:  
$ \frac{dJ(\hat{y})}{dW_2} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) z_1 $  
$ \frac{dJ(\hat{y})}{dW_1} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) f'( W_1 z_0 + \beta_1 ) W_2 z_0 $  
$ \frac{dJ(\hat{y})}{dW_0} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) f'( W_1 z_0 + \beta_1 ) f'( w_0 x_0 + b_0 ) W_2 w_1 x_0 $  

The gradient of the loss w.r.t the biases $\beta$ is the same just without the previous activation $z_{i-1}$ included.  
$ \frac{dJ(\hat{y})}{d\beta_2} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 )$  
$ \frac{dJ(\hat{y})}{d\beta_1} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) f'( W_1 z_0 + \beta_1 ) W_2$  
$ \frac{dJ(\hat{y})}{d\beta_0} = (y_i - \hat{y_i}) f'( W_2 z_1 + \beta_2 ) f'( W_1 z_0 + \beta_1 ) f'( w_0 x_0 + b_0 ) W_2 w_1$  

This means that we need to store both the imediate activation $Z$ e.g. $f(W_i z_{i-1} + \beta_i)$ and all the partials that look like $f'(W_i z_{i-1} + \beta_i)$ for each layer after layer $i$.  
Symbolically this means:  
<!-- let $i$ be the current layer   -->
<!-- let $F'_i $ be the set of partials needed to be stored for layer $i$   -->
<!-- $ \{ F'_i | i \epsilon W, f(W_i z_{i-1} + \beta_i) \} $  -->


In [ ]:
class Activation:
    def __init__(self):
        pass

    def act(self, arr):
        return arr

    def grd(self, arr):
        return np.ones(arr.shape)

In [ ]:
class Sigmoid(Activation):
    def __init__(self, clip=10):
        self.clip = clip

    def act(self, arr):
        arr_ = np.clip(arr, -self.clip, self.clip)
        # return np.power(1 + np.exp(-arr), -1)
        return np.power(1 + np.exp(-arr_), -1)

    def grd(self, arr):
        arr_ = np.clip(arr, -self.clip, self.clip)
        # return np.exp(-arr) * np.power(1 + np.exp(-arr), -2)
        e_neg_x = np.exp(-arr_)
        return e_neg_x * np.power(1 + e_neg_x, -2)

In [ ]:
from typing import List


class StackedNonLinReg:
    def __init__(self, n_layers, learning_rate=0.001):
        # Intialiaze learning rate.
        self.learning_rate = learning_rate

        # Intialize weights and biases.
        self.W = np.random.random(n_layers)
        self.B = np.random.random(n_layers)

        # Initialize activations
        self.activations: List[Activation] = [
            Activation() if (i != 1) else Sigmoid() for i in range(n_layers)
        ]

        # Initialize loss.
        self.j = np.inf

        # Initialize gradients.
        self.dj_wrt_dW = np.zeros(n_layers)
        self.dj_wrt_dB = np.zeros(n_layers)

    # fwd computes each foward step through network.
    def fwd(self, x):
        # Set up the activations setting the 0th activation to be the input data.
        z = [None for _ in range(self.n_layers + 1)]
        z[0] = x
        # Calculate each activation by doing activation of W * z + B.
        for i in range(self.n_layers):
            activation = self.activations[i]
            z[i + 1] = activation.act(z[i] * self.W[i] + self.B[i])
        return z

    # bwd computes the gradient of the cost function given z (forward propogation
    # vector) and y.
    def bwd(self, z, y):
        # Compute loss and partial loss.
        self.j = np.power(y - z[-1], 2).mean()
        loss_partial = y - z[-1]

        # Calculate partial gradients: f'(W_i * z_(i-1)+b_i) for each layer.
        part_grd = [None for _ in range(self.n_layers)]
        for i in range(self.n_layers):
            part_grd[i] = self.activations[i].grd(z[i])

        # z_hat is what the activation should have been.
        for i in range(self.n_layers):
            # Take the prodect of partial gradients activations [0-i].
            part_grd_prod = np.ones(part_grd[0].shape)
            for j in range(i + 1):
                part_grd_prod = part_grd_prod * part_grd[i]

            # Mutliply weights [i+1-n_layers] together.
            weight_prod = 1
            for j in range(i + 1, self.n_layers):
                weight_prod = weight_prod * self.W[i]

            # Calculate the gradient w.r.t. wieghts and biases
            dj_wrt_dB = part_grd_prod * weight_prod * loss_partial
            self.dj_wrt_dB[i] = dj_wrt_dB.mean()
            self.dj_wrt_dW[i] = (dj_wrt_dB * z[0]).mean()

    # step updates the weights and biases based from the gradient.
    def step(self):
        self.W += self.learning_rate * self.dj_wrt_dW
        self.B += self.learning_rate * self.dj_wrt_dB

    @property
    def n_layers(self):
        return len(self.W)

In [ ]:
np.random.seed(0)
stacked_non_lin_reg = StackedNonLinReg(5, learning_rate=0.008)
train(stacked_non_lin_reg, ss_x(x_train), ss_y(y_train), 400, early_stopping=0)

In [ ]:
hv.Layout(
    [
        pred_plot(
            x_train, y_train, ss_x, ss_y_inv, "Train Predictions", stacked_non_lin_reg
        ),
        pred_plot(
            x_test, y_test, ss_x, ss_y_inv, "Test Predictions", stacked_non_lin_reg
        ),
        pred_plot(x, y, ss_x, ss_y_inv, "All Predictions", stacked_non_lin_reg),
    ]
)

### Now to Stack Layers that have Multiple Dimensions to Them

In [ ]:
class RELU(Activation):
    def __init__(self):
        pass

    def act(self, arr):
        return np.maximum(0, arr)

    def grd(self, arr):
        return (arr != 0)

In [ ]:
class LeakyRELU(Activation):
    def __init__(self, neg_slope=0.05):
        self.neg_slope = neg_slope

    def act(self, arr):
        lt_z = (arr < 0)
        return (lt_z * self.neg_slope + (~lt_z)) * arr
        

    def grd(self, arr):
        lt_z = arr < 0
        return lt_z * self.neg_slope +  (~lt_z)

In [ ]:
from typing import List


class StackedMultiNonLinReg:
    def __init__(
        self,
        layers: List[int],
        learning_rate=0.001,
    ):
        # Intialize learning rate.
        self.learning_rate = learning_rate

        # Initialize weights and biases.
        self.B = [np.random.random(layer) for layer in layers[1:]]
        self.W = [
            np.random.random((layers[i], layers[i + 1])) for i in range(len(layers) - 1)
        ]

        # Initialize activations
        self.activations: List[Activation] = [
            Activation() for i in range(self.n_layers)
#             Activation() if (i != 2) else Sigmoid() for i in range(self.n_layers)
        ]

        # Initialize loss.
        self.j = np.inf

        # Initialize gradients.
        self.dj_wrt_dW = [np.zeros(W.shape) for W in self.W]
        self.dj_wrt_dB = [np.zeros(B.shape) for B in self.B]

    # fwd computes each foward step through network.
    def fwd(self, x):
        # Set up the activations setting the 0th activation to be the input data.
        z = [None for _ in range(self.n_layers + 1)]
        z[0] = x
        # Calculate each activation by doing activation of W * z + B.
        for i in range(self.n_layers):
            activation = self.activations[i]
            z[i + 1] = activation.act(np.matmul(z[i], self.W[i]) + self.B[i])
        return z

    # bwd computes the gradient of the cost function given z (forward propogation
    # vector) and y.
    def bwd(self, z, y):
        # Compute loss and partial loss.
        self.j = np.power(y - z[-1], 2).mean()
        loss_partial = y - z[-1]

        # Calculate partial gradients: f'(W_i * z_(i-1)+b_i) for each layer.
        part_grd = [None for _ in range(self.n_layers)]
        for i in range(self.n_layers):
            part_grd[i] = self.activations[i].grd(z[i])

        # z_hat is what the activation should have been.
        for i in range(self.n_layers):
            # Take the prodect of partial gradients activations [0-i].
            part_grd_prod = np.ones(part_grd[0].shape)
            for j in range(i + 1):
                part_grd_prod = part_grd_prod * part_grd[i]

            # Mutliply weights [i+1-n_layers] together.
            weight_prod = np.ones((self.W[i].shape[0], 1))
            for j in range(i + 1, self.n_layers):
                weight_prod = weight_prod * self.W[i]

            # Calculate the gradient w.r.t. wieghts and biases
            dj_wrt_dB = np.matmul(part_grd_prod, weight_prod) * loss_partial

            self.dj_wrt_dB[i] = dj_wrt_dB.mean()
            self.dj_wrt_dW[i] = (dj_wrt_dB * z[0]).mean()

    # step updates the weights and biases based from the gradient.
    def step(self):
        for i in range(self.n_layers):
            self.W[i] += self.learning_rate * self.dj_wrt_dW[i]
            self.B[i] += self.learning_rate * self.dj_wrt_dB[i]

    @property
    def n_layers(self):
        return len(self.W)

In [ ]:
np.random.seed(0)
stacked_multi_non_lin_reg = StackedMultiNonLinReg([1, 2, 1], learning_rate=0.1)

print("Weights Shapes:", list(map(lambda x: x.shape, stacked_multi_non_lin_reg.W)))
print("Activations Shapes:", stacked_multi_non_lin_reg.activations)

train(
    stacked_multi_non_lin_reg,
    ss_x(x_train)[..., np.newaxis],
    ss_y(y_train)[..., np.newaxis],
    25,
)

In [ ]:
hv.Layout(
    [
        pred_plot(
            x_train[..., np.newaxis],
            y_train[..., np.newaxis],
            ss_x,
            ss_y_inv,
            "Train Predictions",
            stacked_multi_non_lin_reg,
        ),
        pred_plot(
            x_test[..., np.newaxis],
            y_test[..., np.newaxis],
            ss_x,
            ss_y_inv,
            "Test Predictions",
            stacked_multi_non_lin_reg,
        ),
        pred_plot(
            x[..., np.newaxis],
            y[..., np.newaxis],
            ss_x,
            ss_y_inv,
            "All Predictions",
            stacked_multi_non_lin_reg,
        ),
    ]
)